In this project, it is aimed to find similar books according to their contents. Goodreads book dataset was used. The similarities were calculated by TF-IDF (term frequency- inverse document frequency) and cosine similarity.

The method is to vectorize the words in each row of the related column in a matrix, i.e TF-IDF matrix here. CountVectorizer matrix also can be used, however it can cause a bias in favor of the words having high frequency. TF-IDF normalizes this bias.





In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
pd.set_option("display.max_columns", None)


In [3]:
df = pd.read_csv(r"C:\Users\PC\Documents\books_enriched.csv")

In [4]:
df.shape

(10000, 30)

In [5]:
df.head(20)

,Unnamed: 0,index,authors,average_rating,best_book_id,book_id,books_count,description,genres,goodreads_book_id,image_url,isbn,isbn13,language_code,original_publication_year,original_title,pages,publishDate,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,ratings_count,small_image_url,title,work_id,work_ratings_count,work_text_reviews_count,authors_2
0,0,0,['Suzanne Collins'],4.34,2767052,1,272,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"['young-adult', 'fiction', 'fantasy', 'science...",2767052,https://images.gr-assets.com/books/1447303603m...,439023483,9.780439e+12,eng,2008.0,The Hunger Games,374.0,09/14/08,66715,127936,560092,1481305,2706317,4780653,https://images.gr-assets.com/books/1447303603s...,"The Hunger Games (The Hunger Games, #1)",2792775,4942365,155254,['Suzanne Collins']
1,1,1,"['J.K. Rowling', 'Mary GrandPré']",4.44,3,2,491,Harry Potter's life is miserable. His parents ...,"['fantasy', 'fiction', 'young-adult', 'classics']",3,https://images.gr-assets.com/books/1474154022m...,439554934,9.780440e+12,eng,1997.0,Harry Potter and the Philosopher's Stone,309.0,11/01/03,75504,101676,455024,1156318,3011543,4602479,https://images.gr-assets.com/books/1474154022s...,Harry Potter and the Sorcerer's Stone (Harry P...,4640799,4800065,75867,"['J.K. Rowling', 'Mary GrandPré']"
2,2,2,['Stephenie Meyer'],3.57,41865,3,226,About three things I was absolutely positive.\...,"['young-adult', 'fantasy', 'romance', 'fiction...",41865,https://images.gr-assets.com/books/1361039443m...,316015849,9.780316e+12,eng,2005.0,Twilight,501.0,09/06/06,456191,436802,793319,875073,1355439,3866839,https://images.gr-assets.com/books/1361039443s...,"Twilight (Twilight, #1)",3212258,3916824,95009,['Stephenie Meyer']
3,3,3,['Harper Lee'],4.25,2657,4,487,The unforgettable novel of a childhood in a sl...,"['classics', 'fiction', 'historical-fiction', ...",2657,https://images.gr-assets.com/books/1361975680m...,61120081,9.780061e+12,eng,1960.0,To Kill a Mockingbird,324.0,05/23/06,60427,117415,446835,1001952,1714267,3198671,https://images.gr-assets.com/books/1361975680s...,To Kill a Mockingbird,3275794,3340896,72586,['Harper Lee']
4,4,4,['F. Scott Fitzgerald'],3.89,4671,5,1356,Alternate Cover Edition ISBN: 0743273567 (ISBN...,"['classics', 'fiction', 'historical-fiction', ...",4671,https://images.gr-assets.com/books/1490528560m...,743273567,9.780743e+12,eng,1925.0,The Great Gatsby,200.0,09/28/04,86236,197621,606158,936012,947718,2683664,https://images.gr-assets.com/books/1490528560s...,The Great Gatsby,245494,2773745,51992,['F. Scott Fitzgerald']
5,5,5,['John Green'],4.26,11870085,6,226,Despite the tumor-shrinking medical miracle th...,"['young-adult', 'romance', 'fiction', 'contemp...",11870085,https://images.gr-assets.com/books/1360206420m...,525478817,9.780525e+12,eng,2012.0,The Fault in Our Stars,313.0,01/10/12,47994,92723,327550,698471,1311871,2346404,https://images.gr-assets.com/books/1360206420s...,The Fault in Our Stars,16827462,2478609,140739,['John Green']
6,6,6,['J.R.R. Tolkien'],4.25,5907,7,969,In a hole in the ground there lived a hobbit. ...,"['fantasy', 'classics', 'fiction', 'young-adult']",5907,https://images.gr-assets.com/books/1372847500m...,618260307,9.780618e+12,eng,1937.0,The Hobbit or There and Back Again,366.0,08/15/02,46023,76784,288649,665635,1119718,2071616,https://images.gr-assets.com/books/1372847500s...,The Hobbit,1540236,2196809,37653,['J.R.R. Tolkien']
7,7,7,['J.D. Salinger'],3.79,5107,8,360,The hero-narrator of The Catcher in the Rye is...,"['classics', 'fiction', 'young-adult']",5107,https://images.gr-assets.com/books/1398034300m...,316769177,9.780317e+12,eng,1951.0,The Catcher in the Rye,277.0,01/30/01,109383,185520,455042,661516,709176,2044241,https://images.gr-assets.com/books/1398034300s...,The Catcher in the Rye,3036731,2120637,44920,['J.D. Salinger']
8,8,8,['Dan Brown'],3.85,960,9,311,World-renowned Harvard symbologist Robert Lang...,"['fiction', 'mystery', 'thriller', 'suspense',...",960,https://images.gr-assets.

In [6]:
df = df.drop("Unnamed: 0", axis=1)

In [7]:
# The 'description' column gives idea about the content, so this will be used. Some rows contain unrelevant information, so
# they are removed.

df["description"].str.contains("ISBN", na=False).value_counts()

False    9838
True      162
Name: description, dtype: int64

In [8]:
df["description"].str.contains("cover edition", na=False).value_counts()

False    9790
True      210
Name: description, dtype: int64

In [9]:
df = df[~df["description"].str.contains("ISBN", na=False)]

In [10]:
df = df[~df["description"].str.contains("cover edition", na=False)]

In [11]:
df["description"].head()

0    WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...
1    Harry Potter's life is miserable. His parents ...
2    About three things I was absolutely positive.\...
3    The unforgettable novel of a childhood in a sl...
5    Despite the tumor-shrinking medical miracle th...
Name: description, dtype: object

In [12]:
# String operations are done before applying cosine similarity.

df['description'] = df['description'].str.lower()
df['description'] = df['description'].str.replace(r'[^\w\s]+', ' ')
df['description'] = df['description'].str.replace('\n', ' ')

C:\Users\PC\AppData\Local\Temp/ipykernel_5280/299666640.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['description'] = df['description'].str.replace(r'[^\w\s]+', ' ')


In [13]:
# stop_words are commonly used but have no meaning, like 'is', 'the', etc.

tfidf = TfidfVectorizer(stop_words = 'english')

In [14]:
df['description'].isna().sum()

57

In [15]:
df['description'] = df['description'].fillna(' ')

In [16]:
df['description'].head(20)



0     winning means fame and fortune losing means ce...
1     harry potter s life is miserable  his parents ...
2     about three things i was absolutely positive  ...
3     the unforgettable novel of a childhood in a sl...
5     despite the tumor shrinking medical miracle th...
6     in a hole in the ground there lived a hobbit  ...
7     the hero narrator of the catcher in the rye is...
8     world renowned harvard symbologist robert lang...
10    the unforgettable  heartbreaking story of the ...
11    in beatrice prior s dystopian chicago world  s...
12    discovered in the attic in which she spent the...
14    sparks are igniting flames are spreading and t...
15    harry potter s third year at hogwarts is full ...
16    the final book in the ground breaking hunger g...
17    there is a door at the end of a silent corrido...
18     my name was salmon  like the fish  first name...
19    ever since harry potter had come home for the ...
20    harry potter is midway through his trainin

In [17]:
tfidf_matrix = tfidf.fit_transform(df["description"])

In [18]:
# Similarity matrix is generated using cosine similarity.

cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [19]:
cos_sim

array([[1.        , 0.02322108, 0.01937285, ..., 0.01476556, 0.00422354,
        0.00959166],
       [0.02322108, 1.        , 0.        , ..., 0.03664043, 0.01163007,
        0.01411476],
       [0.01937285, 0.        , 1.        , ..., 0.        , 0.        ,
        0.00587242],
       ...,
       [0.01476556, 0.03664043, 0.        , ..., 1.        , 0.01077162,
        0.02375643],
       [0.00422354, 0.01163007, 0.        , ..., 0.01077162, 1.        ,
        0.00441835],
       [0.00959166, 0.01411476, 0.00587242, ..., 0.02375643, 0.00441835,
        1.        ]])

In [20]:
# In order to find the similar books, first books are ordered by their indices.

indices = pd.Series(df.index, index=df["title"])

In [21]:
indices

title
The Hunger Games (The Hunger Games, #1)                        0
Harry Potter and the Sorcerer's Stone (Harry Potter, #1)       1
Twilight (Twilight, #1)                                        2
To Kill a Mockingbird                                          3
The Fault in Our Stars                                         5
                                                            ... 
Anastasia Krupnik (Anastasia Krupnik, #1)                   9994
Feverborn (Fever, #8)                                       9996
Breaking Point (Tom Clancy's Net Force, #4)                 9997
Chasing The Night (Eve Duncan, #11; Catherine Ling, #1)     9998
The First World War                                         9999
Length: 9691, dtype: int64

In [22]:
indices.shape

(9691,)

In [23]:
# Duplicated values are removed.

indices = indices[~indices.index.duplicated(keep='last')]

In [24]:
indices.shape

(9656,)

In [25]:
# A random book is chosen, Pet Sematary by Stephen King.

book_index = indices["The Green Mile"]

In [26]:
# The similarity matrix of the chosen book.

cos_sim[book_index]

array([0.01025581, 0.01809545, 0.0038223 , ..., 0.        , 0.        ,
       0.00956674])

In [27]:
# To get a better view, the matrix is converted to a dataframe.

similarity_scores = pd.DataFrame(cos_sim[book_index], columns = ['score'])

In [28]:
similarity_scores

,score
0,0.010256
1,0.018095
2,0.003822
3,0.034342
4,0.006334
...,...
9686,0.000000
9687,0.002570
9688,0.000000
9689,0.000000


In [29]:
# Most similar 20 books are chosen.

most_similar= similarity_scores.sort_values(by='score', ascending=False)[1:21]

In [30]:
book_indices = most_similar.index

In [31]:
# The similar books to the chosen book, with their titles and genres

similar_books = df[['title', 'genres']].iloc[book_indices]

In [32]:
similar_books

,title,genres
6014,Never Knowing,"['thriller', 'mystery', 'fiction', 'suspense',..."
3232,When It Happens,"['young-adult', 'romance', 'contemporary', 'ch..."
84,The Secret Garden,"['classics', 'fiction', 'young-adult', 'histor..."
4836,"Undone (Will Trent, #3)","['thriller', 'mystery', 'crime', 'fiction', 's..."
5431,"Beyond Reach (Grant County, #6)","['mystery', 'thriller', 'crime', 'fiction', 's..."
6429,"Unseen (Will Trent, #7)","['mystery', 'thriller', 'crime', 'fiction', 's..."
2223,"Blindsighted (Grant County, #1)","['mystery', 'thriller', 'crime', 'fiction', 's..."
4794,"A Faint Cold Fear (Grant County, #3)","['mystery', 'thriller', 'crime', 'fiction', 's..."
2407,"Beautiful Stranger (Beautiful Bastard, #2)","['romance', 'contemporary', 'chick-lit', 'fict..."
4949,"Kisscut (Grant County, #2)","['mystery', 'thriller', 'crime', 'fiction', 's..."


In [33]:
df[df["title"] == "The Green Mile"]

,index,authors,average_rating,best_book_id,book_id,books_count,description,genres,goodreads_book_id,image_url,isbn,isbn13,language_code,original_publication_year,original_title,pages,publishDate,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,ratings_count,small_image_url,title,work_id,work_ratings_count,work_text_reviews_count,authors_2
450,450,['Stephen King'],4.42,11566,488,162,when it first appeared one volume per month ...,"['horror', 'fiction', 'fantasy', 'thriller', '...",11566,https://images.gr-assets.com/books/1373903563m...,451933028,9.780452e+12,eng,1996.0,The Green Mile,592.0,09/01/96,725,3044,19770,59813,107757,173950,https://images.gr-assets.com/books/1373903563s...,The Green Mile,15599,191109,4873,['Stephen King']


In [34]:
# The dataframe of the similar books with The Green Mile

similar_books_df = pd.merge(similar_books[['title', 'genres']], df, how='left', on='title')

In [35]:
similar_books_df

,title,genres_x,index,authors,average_rating,best_book_id,book_id,books_count,description,genres_y,goodreads_book_id,image_url,isbn,isbn13,language_code,original_publication_year,original_title,pages,publishDate,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,ratings_count,small_image_url,work_id,work_ratings_count,work_text_reviews_count,authors_2
0,Never Knowing,"['thriller', 'mystery', 'fiction', 'suspense',...",6014,['Chevy Stevens'],3.85,10169662,6907,56,at thirty three sara gallagher is finally happ...,"['thriller', 'mystery', 'fiction', 'suspense',...",10169662,https://images.gr-assets.com/books/1294615110m...,NaN,9.780313e+12,eng,2011.0,Never Knowing,410.0,July 5th 2011,373,1155,4452,7114,5079,16178,https://images.gr-assets.com/books/1294615110s...,13723008,18173,2489,['Chevy Stevens']
1,When It Happens,"['young-adult', 'romance', 'contemporary', 'ch...",3232,['Susane Colasanti'],3.74,781046,3487,26,at the start of her senior year in high school...,"['young-adult', 'romance', 'contemporary', 'ch...",781046,https://images.gr-assets.com/books/1268350521m...,670060291,9.780670e+12,eng,2006.0,When It Happens,287.0,May 4th 2006,1092,3139,9215,9979,10109,32443,https://images.gr-assets.com/books/1268350521s...,767067,33534,1226,['Susane Colasanti']
2,The Secret Garden,"['classics', 'fiction', 'young-adult', 'histor...",84,['Frances Hodgson Burnett'],4.12,2998,93,1350,one of the most delightful and enduring class...,"['classics', 'fiction', 'young-adult', 'histor...",2998,https://images.gr-assets.com/books/1327873635m...,517189607,9.780517e+12,eng,1911.0,The Secret Garden,331.0,09/01/98,8005,25207,129646,243821,285570,639357,https://images.gr-assets.com/books/1327873635s...,3186437,692249,13054,['Frances Hodgson Burnett']
3,"Undone (Will Trent, #3)","['thriller', 'mystery', 'crime', 'fiction', 's...",4836,['Karin Slaughter'],4.24,5764902,5408,84,when a tortured young woman enters the trauma ...,"['thriller', 'mystery', 'crime', 'fiction', 's...",5764902,https://s.gr-assets.com/assets/nophoto/book/11...,385341962,9.780385e+12,eng,2009.0,Genesis,436.0,July 14th 2009,101,313,2998,9457,9371,17871,https://s.gr-assets.com/assets/nophoto/book/50...,5936637,22240,1235,['Karin Slaughter']
4,"Beyond Reach (Grant County, #6)","['mystery', 'thriller', 'crime', 'fiction', 's...",5431,['Karin Slaughter'],4.19,21720,6151,60,sara linton resident medical examiner pediatri...,"['mystery', 'thriller', 'crime', 'fiction', 's...",21720,https://s.gr-assets.com/assets/nophoto/book/11...,038533947X,9.780385e+12,eng,2007.0,Beyond Reach,404.0,July 31st 2007,211,515,2881,6934,7986,15359,https://s.gr-assets.com/assets/nophoto/book/50...,1952659,18527,985,['Karin Slaughter']
5,"Unseen (Will Trent, #7)","['mystery', 'thriller', 'crime', 'fiction', 's...",6429,['Karin Slaughter'],4.17,16071767,7474,46,karin slaughter s novels featuring special age...,"['mystery', 'thriller', 'crime', 'fiction', 's...",16071767,https://images.gr-assets.com/books/1361517437m...,345539478,9.780346e+12,eng,2013.0,Unseen,379.0,July 2nd 2013,113,392,2532,6612,6402,12277,https://images.gr-assets.com/books/1361517437s...,23590922,16051,1226,['Karin Slaughter']
6,"Blindsighted (Grant County, #1)","['mystery', 'thriller', 'crime', 'fiction', 's...",2223,['Karin Slaughter'],4.10,21718,2373,96,a small georgia town erupts in panic when a yo...,"['mystery', 'thriller', 'crime', 'fiction', 's...",21718,https://s.gr-assets.com/assets/nophoto/book/11...,380820889,9.780381e+12,eng,2001.0,Blindsighted,418.0,October 1st 2002,577,1487,8425,18229,17949,41857,https://s.gr-assets.com/assets/nophoto/book/50...,2590340,46667,1923,['Karin Slaughter']
7,"A Faint Cold Fear (Grant County, #3)","['mystery', 'thriller', 'crime', 'fiction', 's...",4794,['Karin Slaughter'],4.12,21721,5359,79,the third pulse pounding novel in the grant co...,"['mystery', 'thriller', 'crime', 'fiction', 's...",21721,https://images.gr-assets.com/books/1394126332m...,60534052,9.780061e+12,eng,2003.0,A Faint Cold Fear,422.0,Ju